# File format analysis

In order to approach the project with a broader scope, we have not only decided to focus on the data analysis component but also some aspects regarding the data management of a possible data exploitation process this project could be encapsulated into.

We have defined different data zones to store the different steps of the data analysis process. Although we want the solution to be very general to fit most cases, we will also try to propose suitable approach to storing this data.

We have seen that lots of the values to be stored are repeated (i.e. the measurement unit of energy which is the same each row), so it makes sense to use some kind of file format that uses compression. For this reason we have decided to test parquet.

In [19]:
import pandas as pd
import time
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

filepath = '../data/test_file_format/gen_DE_B18'

# Create an empty DataFrame with columns
columns = ['Reading time (s)', 'Storage space (KB)']
df_comparison = pd.DataFrame(columns=columns)

# Reading a trivial original data file
df = pd.read_csv(f'{filepath}.csv', sep=',', decimal='.', encoding='utf-8')
df.head(5)

# Setting the correct types
df.StartTime = pd.to_datetime(df.StartTime, format='%Y-%m-%dT%H:%M%zZ')
df.EndTime = pd.to_datetime(df.EndTime, format='%Y-%m-%dT%H:%M%zZ')
df.AreaID = df.AreaID.astype(str)
df.UnitName = df.UnitName.astype(str)
df.PsrType = df.PsrType.astype(str)
df.quantity = df.quantity.astype(int)

# Creating some test files
compression_algorithms = ['none', 'snappy', 'gzip', 'brotli']
df.to_csv(f'{filepath}_csv.csv', index = False, encoding = 'utf-8', sep = ',')
for ca in compression_algorithms:
    df.to_parquet(f'{filepath}_parquet_{ca}.parquet', index = False, engine = 'pyarrow', compression = ca)

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2021-12-31T23:45+00:00Z,2022-01-01T00:00+00:00Z,10Y1001A1001A83F,MAW,B18,5688
1,2022-01-01T00:00+00:00Z,2022-01-01T00:15+00:00Z,10Y1001A1001A83F,MAW,B18,5795
2,2022-01-01T00:15+00:00Z,2022-01-01T00:30+00:00Z,10Y1001A1001A83F,MAW,B18,5775
3,2022-01-01T00:30+00:00Z,2022-01-01T00:45+00:00Z,10Y1001A1001A83F,MAW,B18,5843
4,2022-01-01T00:45+00:00Z,2022-01-01T01:00+00:00Z,10Y1001A1001A83F,MAW,B18,5699


Although we are duplicating data, for completion purposes we will store the CSV again and check some metrics.

In [20]:
# Reading the test files and storing the time
start_time = time.time()
df_csv = pd.read_csv(f'{filepath}.csv', sep=',', decimal='.', encoding='utf-8')
end_time = time.time()

# Calculate reading time
reading_time = end_time - start_time
    
# Get file size in KB
file_size_bytes = os.path.getsize(f'{filepath}.csv')
file_size_kb = file_size_bytes / 1024  # Convert bytes to KB

comparison_data = []

# Appending data to the comparison dictionary
comparison_data.append({
    'File format': 'CSV',
    'Reading time (s)': reading_time,
    'Storage space (KB)': file_size_kb
})

We will also check some metrics for the parquet format using different compression algorithms

In [21]:

# Iterating over the different parquet options
for ca in compression_algorithms:
    file = f'{filepath}_parquet_{ca}.parquet'
        
    start_time = time.time()
    df = pd.read_parquet(file, engine = 'pyarrow')
    end_time = time.time()
    
    # Calculate reading time
    reading_time = end_time - start_time
    
    # Get file size in KB
    file_size_bytes = os.path.getsize(file)
    file_size_kb = file_size_bytes / 1024  # Convert bytes to KB
    
    # Append data to the comparison dictionary
    comparison_data.append({
        'File format': f'Parquet - {ca}',
        'Reading time (s)': reading_time,
        'Storage space (KB)': file_size_kb
    })

# Create a DataFrame from the list of dictionaries
df_comparison = pd.DataFrame(comparison_data)

df_comparison['Ratio (seconds)/(KB)'] = df_comparison['Reading time (s)']/df_comparison['Storage space (KB)']
df_comparison

,File format,Reading time (s),Storage space (KB),Ratio (seconds)/(KB)
0,CSV,0.034355,2659.800781,0.000013
1,Parquet - none,0.034975,796.294922,0.000044
2,Parquet - snappy,0.005335,718.826172,0.000007
3,Parquet - gzip,0.005535,571.174805,0.000010
4,Parquet - brotli,0.004627,499.117188,0.000009


In this small example, we can see how CSV is greatly inferior in terms of both reading speed and storage space. In addition, parquet stores the types of the different columns, so there is no apparent downside to using it for this project. In particular we have decided to use snappy as it has a better ratio compared to the other checked options.